In [1]:
import os
import numpy as np
import pandas as pd
import flopy
import matplotlib
import matplotlib.pyplot as plt

In [2]:
########### User Input Variables ###############
input_ws = 'ModelFiles'
png_ws = '_PNGs'
grbFile = 'CD_00.dis.grb'
num_Layers = 37
num_Nodes = 257224
################################################

In [3]:
#Establish model grid from binary grid file
grbPath = os.path.join(input_ws, grbFile)
grd = flopy.mf6.utils.binarygrid_util.MfGrdFile(grbPath, verbose=False)
mg = grd.modelgrid

In [4]:
#Create list of K array files
k_arr_ws = os.path.join('.', 'KArrays')
arr_list = [x[:-4] for x in os.listdir(k_arr_ws) if x[-4:]=='.dat']
arr_list

['kh_Avg', 'kh_Calibration', 'kh_StdDev']

In [5]:
#Create a list of layer numbers based on the total number of layers
layer_list = []
l = 0
while l < num_Layers:
    layer_list.append(l+1)
    l += 1

In [6]:
#Create list of last node in layer
n = 0
nodes_per_l = num_Nodes/num_Layers
node_list = []
count = 0
while n < num_Nodes:
    if count == nodes_per_l:
        node_list.append(n)
        count = 0
    count += 1
    n += 1
node_list.append(num_Nodes)

6952.0


[6952,
 13904,
 20856,
 27808,
 34760,
 41712,
 48664,
 55616,
 62568,
 69520,
 76472,
 83424,
 90376,
 97328,
 104280,
 111232,
 118184,
 125136,
 132088,
 139040,
 145992,
 152944,
 159896,
 166848,
 173800,
 180752,
 187704,
 194656,
 201608,
 208560,
 215512,
 222464,
 229416,
 236368,
 243320,
 250272,
 257224]

In [9]:
for k in arr_list:
    file = open(os.path.join(k_arr_ws, k + '.dat'), 'r')
    vals = file.readlines()
    vals_arr = np.array(vals).astype(float)
    currParam = k.split('_')[0]
    currStat = k.split('_')[1]
    print(currParam, currStat)
    if currParam == 'kh':
        bounds = [1.0e-6, 1.0e-5, 5.0e-5, 1.0e-4, 5.0e-4, 1.0e-3, 5.0e-3, 1.0e-2, 5.0e-2, 1.0e-1]
        rgb = [[102,0,0],[204,0,0],[255,128,0],[255,255,102],[128,128,128],[102,255,102],[0,255,255],[0,0,255],[76,0,153]]
        rgb=np.array(rgb)/255.0
        cmap = matplotlib.colors.ListedColormap(rgb,"")
        norm = matplotlib.colors.BoundaryNorm(bounds, 10)
        frmt = '%.2E'
        cbar_label = 'Kh (cm/s)'
    elif currParam == 'kz':
        bounds = [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]
        rgb = [[102,0,0],[255,128,0],[128,128,128],[0,0,255],[76,0,153]]
        rgb=np.array(rgb)/255.0
        cmap = matplotlib.colors.ListedColormap(rgb,"")
        norm = matplotlib.colors.BoundaryNorm(bounds, 6)
        frmt = '%.2f'
        cbar_label = 'Kh:Kz'
              
    rgb=np.array(rgb)/255.0 
    
    prevNode = 0
    layer = 1
    for node in node_list:

        if currStat == 'Calibration':
            plt_title = '{0} Distribution from Calibration Run in Layer {1}'.format(currParam, layer)
            pngName = os.path.join(png_ws, '{0}_L{1:0=3d}_Calibration'.format(currParam, layer) + '.png')
        elif currStat == 'Avg':
            plt_title = 'Average {0} Distribution of IES Realizations in Layer {1}'.format(currParam, layer)
            pngName = os.path.join(png_ws, '{0}_L{1:0=3d}_IESAvg'.format(currParam, layer) + '.png')
        elif currStat == 'StdDev':
            plt_title = 'Standard Deviation of {0} Distribution of IES Realizations in Layer {1}'.format(currParam, layer)
            pngName = os.path.join(png_ws, '{0}_L{1:0=3d}_IESStdDev'.format(currParam, layer) + '.png')
        
        currNode = node
        plt_arr = vals_arr[prevNode:currNode]
        fig = plt.figure(figsize=(10, 10), constrained_layout = True)
        ax = fig.add_subplot(1, 1, 1, aspect='equal')
        ax.set_title(plt_title)
        mm = flopy.plot.PlotMapView(modelgrid=mg, ax=ax, layer = 0)
        mm.plot_grid(edgecolor='black', linewidths = 0.05)
        arr = mm.plot_array(a=plt_arr, cmap = cmap, norm = norm)
        cbar = fig.colorbar(arr, shrink = 0.55, format = frmt)
        cbar.set_label(cbar_label, rotation = 270, labelpad = 20)
        
        print('Saving {0}'.format(pngName))
        plt.savefig(pngName, pad_inches = 0.01)
        plt.close()
        
        layer += 1
        prevNode = currNode

<ipython-input-9-2e93c16db325>:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  vals_arr = np.array(vals).astype(np.float)


IndexError: list index out of range